In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, desc
jars = [
    "/home/jovyan/work/jars/postgresql-42.7.5.jar",
    "/home/jovyan/work/jars/commons-pool2-2.12.0.jar",
]
spark = SparkSession.builder \
    .appName("Loading Process") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", '4G') \
    .config("spark.executor.cores", '4') \
    .config("spark.executor.instances", "1") \
    .config("spark.cores.max", '4') \
    .config("spark.deploy.defaultCores", "4")   \
    .config("spark.task.cpus", "4") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.jars", ",".join(jars)) \
    .config("spark.sql.session.timeZone", "Asia/Ho_Chi_Minh") \
    .config("spark.executorEnv.TZ", "Asia/Ho_Chi_Minh") \
    .config("spark.hadoop.fs.defaultFS","hdfs://namenode:9000") \
    .getOrCreate()

25/05/04 04:52:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
from datetime import datetime
from zoneinfo import ZoneInfo

now_vietnam = datetime.now(ZoneInfo("Asia/Ho_Chi_Minh"))
current_date = now_vietnam.strftime("%Y-%m-%d")

In [4]:
df = spark.read.parquet(f"hdfs://namenode:9000/data/weather/date={current_date}").orderBy(desc("timestamp"))

In [4]:
from pyspark.sql import functions as F

In [5]:
    # Chọn và chuyển đổi các trường cần thiết
cities_df = df.select(
    "city_id",
    "city_name",
    "latitude",
    "longitude"
).distinct()
cities_df.show()

+-------+----------------+--------+---------+
|city_id|       city_name|latitude|longitude|
+-------+----------------+--------+---------+
|1581298|        Haiphong|   20.87|   106.68|
|1566083|Ho Chi Minh City|   10.78|    106.7|
|1567788|       Soc Trang|     9.6|   105.97|
|1581130|           Hanoi|   21.03|    105.8|
|1580240|             Hue|   16.46|   107.59|
+-------+----------------+--------+---------+



In [6]:
cities_df.printSchema()

root
 |-- city_id: integer (nullable = true)
 |-- city_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [ ]:
weather_df = df.select(
        "city_id",
        "timestamp",
        "temperature_celsius",
        "feels_like_celsius",
        "temp_min_celsius",
        "temp_max_celsius",
        "pressure_hpa",
        "humidity_percent",
        "wind_speed_mps",
        "wind_direction_deg",
        "wind_gust_mps",
        "rain_1h_mm",
        "cloud_coverage_percent",
        "weather_condition",
        "weather_description",
        "sunrise_time",
        "sunset_time"
    )
weather_df.printSchema()

+-------+-------------------+-------------------+------------------+------------------+------------------+------------+----------------+--------------+------------------+-------------+----------+----------------------+-----------------+-------------------+-------------------+-------------------+
|city_id|          timestamp|temperature_celsius|feels_like_celsius|  temp_min_celsius|  temp_max_celsius|pressure_hpa|humidity_percent|wind_speed_mps|wind_direction_deg|wind_gust_mps|rain_1h_mm|cloud_coverage_percent|weather_condition|weather_description|       sunrise_time|        sunset_time|
+-------+-------------------+-------------------+------------------+------------------+------------------+------------+----------------+--------------+------------------+-------------+----------+----------------------+-----------------+-------------------+-------------------+-------------------+
|1580240|2025-05-04 09:45:04|  32.05000000000001| 39.05000000000001| 32.05000000000001| 32.05000000000001|   

In [8]:
# Tính toán các aggregates theo ngày và thành phố   

daily_agg = weather_df.groupBy("city_id").agg(
        # Nhiệt độ
    F.current_date().alias("aggregate_date"),
    F.avg("temperature_celsius").alias("avg_temperature"),
    F.max("temperature_celsius").alias("max_temperature"),
    F.min("temperature_celsius").alias("min_temperature"),
        # Nhiệt độ cảm nhận
    F.avg("feels_like_celsius").alias("avg_feels_like"),
    F.max("feels_like_celsius").alias("max_feels_like"),
    F.min("feels_like_celsius").alias("min_feels_like"),
        # Áp suất
    F.avg("pressure_hpa").alias("avg_pressure"),
    F.max("pressure_hpa").alias("max_pressure"),
    F.min("pressure_hpa").alias("min_pressure"),
        # Độ ẩm
    F.avg("humidity_percent").alias("avg_humidity"),
    F.max("humidity_percent").alias("max_humidity"),
    F.min("humidity_percent").alias("min_humidity"),
        # Gió
    F.avg("wind_speed_mps").alias("avg_wind_speed"),
    F.max("wind_speed_mps").alias("max_wind_speed"),
    F.min("wind_speed_mps").alias("min_wind_speed"),
        # Giật gió
    F.max("wind_gust_mps").alias("max_wind_gust"),
        # Lượng mưa
    F.sum("rain_1h_mm").alias("total_rainfall"),
        # Số lần đo
    F.count("*").alias("measurement_count"),
        # Thời gian cập nhật
    F.current_timestamp().alias("date")
)

In [9]:
daily_agg.show()

+-------+--------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------+------------+------------+------------+------------+------------+--------------+--------------+--------------+-------------+--------------+-----------------+--------------------+
|city_id|aggregate_date|   avg_temperature|   max_temperature|   min_temperature|    avg_feels_like|   max_feels_like|   min_feels_like|avg_pressure|max_pressure|min_pressure|avg_humidity|max_humidity|min_humidity|avg_wind_speed|max_wind_speed|min_wind_speed|max_wind_gust|total_rainfall|measurement_count|                date|
+-------+--------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------+------------+------------+------------+------------+------------+--------------+--------------+--------------+-------------+--------------+-----------------+--------------------+
|1567788|    202